**References** : https://github.com/adwait1291/Nike-Style-Product-Description-Generator-With-GPT3

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
#Load the pre-trained GPT model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# adding padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Belajar Data/iykra/Hands-on/dataset/Dataset for Product Description Generator/Week3NikeProductDescriptionsGenerator.csv")
df.head(3)

,Title,Subtitle,Product Description
0,Nike Air Force 1 '07,Men's Shoes,It doesn't get more legendary than this. Desig...
1,Nike Air Max Dawn SE,Men's Shoes,Find out what moves you with the Air Max Dawn....
2,Nike SB Dunk Low Pro Premium,Skate Shoes,Pack your style—on your feet. Bringing a fresh...


In [ ]:
df.shape

(400, 3)

In [ ]:
df.isna().sum()

,0
Title,0
Subtitle,0
Product Description,0


In [ ]:
df['Subtitle'].unique()

array(["Men's Shoes", 'Skate Shoes',
       "Men's Easy On/Off Road Running Shoes", "Men's Basketball Shoes",
       "Men's Road Running Shoes", "Men's Training Shoes", "Men's Shoe",
       'Easy On/Off Basketball Shoes', "Men's Hard Court Tennis Shoes",
       "Men's Slides", "Men's Road Racing Shoes", 'Skate Shoe',
       "Men's Jacket", "Men's Woven Trousers",
       "Men's Hooded Pullover Jacket", "Men's Mesh Shorts",
       "Men's T-Shirt", "Men's Joggers", "Men's Fleece Trousers",
       "Men's French Terry Pullover Hoodie",
       "Men's All-over Print Basketball T-Shirt",
       "Men's 18cm (approx.) Unlined Versatile Shorts",
       "Men's Short-Sleeve Fitness Top", "Men's Mock-Neck Golf Polo",
       "Men's Short-Sleeve Fitness Tank",
       "Men's 18cm (approx.) 2-in-1 Versatile Shorts",
       "Men's Long-Sleeve T-Shirt",
       "Men's 20cm (approx.) Basketball Shorts", "Men's Fitness T-Shirt",
       "Men's Woven Flow Shorts", "Men's 5cm (approx.) Running Shorts",
       "

In [ ]:
df['Subtitle'].shape

(400,)

In [ ]:
descriptions = df['Product Description']
descriptions.head(3)

,Product Description
0,It doesn't get more legendary than this. Desig...
1,Find out what moves you with the Air Max Dawn....
2,Pack your style—on your feet. Bringing a fresh...


In [ ]:
# tokenize product descriptions
def preprocess(descriptions):
  descriptions = descriptions.tolist()
  encodings = tokenizer(descriptions, truncation=True, padding=True)
  # return tokenizer(descriptions, return_tensors="pt", truncation=True, padding=True)
  return Dataset.from_dict(encodings)

In [ ]:
train_dataset = preprocess(descriptions)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 400
})

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True
)

In [ ]:
# initialize Trainer for fine-tuning model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [ ]:
# fine-tune model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bagasaqmal70 (bagasaqmal70-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=300, training_loss=3.1428269449869792, metrics={'train_runtime': 397.9203, 'train_samples_per_second': 3.016, 'train_steps_per_second': 0.754, 'total_flos': 79612416000000.0, 'train_loss': 3.1428269449869792, 'epoch': 3.0})

In [ ]:
# function to generate response based on a prompt
def generate_response(prompt,max_length=1500):
  print(f"Prompt: {prompt}")
  inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
  print(f"Input shape: {inputs.shape}")
  # outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
  outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, pad_token_id=tokenizer.eos_token_id)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_text

In [ ]:
# example prompt
prompt = "Nike Air Max 90"
print(generate_response(prompt))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt: Nike Air Max 90
Input shape: torch.Size([1, 5])
Nike Air Max 90s are the ultimate sweat-wicking outfit. Made from sweatwicks and sweat repellent fabric, they're made from at least 75% recycled polyester fibres. They're designed to help you stay dry and cool while you work out. Plus, you can see through the mesh to see how your sweat moves. This product is made with at-home approval. For more details, check out our product page.


In [ ]:
# lets play arround with your prompt!
prompt = "Nike SB Dunk Low Pro Premium"
print(generate_response(prompt))

Prompt: Nike SB Dunk Low Pro Premium
Input shape: torch.Size([1, 7])
Nike SB Dunk Low Pro Premium Air Jordan 1 Mid Air Air Max Air is made from at least 75% recycled polyester fibres. This product is designed to be worn with your favourite pair of shoes. The sole is a blend of recycled cotton fibre and recycled synthetic fibrous fibreglass. It's made with at most 50% organic cotton. Weave fabric is soft and breathable so you can wear it everyday. Made from 100% recyclable polyesters, it's a lightweight, breathably-wicking product that's easy to clean. Plus, the mesh on the outside of the shoe is breath-resistant so it won't get in the way you want it to. So, where to next? Let's see!

This product was made in at Least 75%. This is an all-over print. Our print is 100%, so we can't tell you how many times you've made this product.
. For the print, we used a slightly smaller print to help keep the look of our original print close to the original Nike Air cushioning. You can see the printe

In [ ]:
prompt = "Nike Air Force 1 '07"
print(generate_response(prompt))

Prompt: Nike Air Force 1 '07
Input shape: torch.Size([1, 7])
Nike Air Force 1 '07 is a must-have for any skate enthusiast. The Air Max 1 is made from lightweight fabric that's easy to wear and easy on the skin. It's made with a lightweight, breathable fabric with an easy fit that keeps you cool and dry. Plus, it's designed to help you stay dry and cool while you skate. This product is not for use in a roomy room. For more information, see our product label.

This product has a 100% recycled polyester fibres. We made these to keep you dry, dry & cool. They're made to be worn on your skateboard or in your favourite shorts. These are made for skateboarding, so you can skate with them. So, what else are you? We're not going to tell you how to skate, but we're going ahead. Let's get started. Go ahead, get ready. Oh, and do you know what that means? It means you're gonna skate fast. And you'll skate hard. That's what we love. You're ready to go. Now, let's go ahead and get to work. What's yo

In [ ]:
df

,Title,Subtitle,Product Description
0,Nike Air Force 1 '07,Men's Shoes,It doesn't get more legendary than this. Desig...
1,Nike Air Max Dawn SE,Men's Shoes,Find out what moves you with the Air Max Dawn....
2,Nike SB Dunk Low Pro Premium,Skate Shoes,Pack your style—on your feet. Bringing a fresh...
3,Nike Air Force 1 Mid '07 LX,Men's Shoes,The celebrations just keep coming. Unbox the A...
4,Nike Air Force 1 Mid '07,Men's Shoes,"Got your fave colour yet? No worries, the Colo..."
...,...,...,...
395,Nike Yoga Dri-FIT ADV,Women's High-Rise 18cm (approx.) Shorts,"Enjoy the soft, breathable experience of the N..."
396,Nike Epic Fast,Women's Mid-Rise Running Leggings,"Keep running, with the Nike Epic Fast Mid-rise..."
397,Nike Sportswear A.I.R. Essential,Older Kids' (Girls') Mid-Rise Leggings,We teamed up with Chicago-based artist Cody Hu...
398,Nike ACG Dri-FIT One,Older Kids' (Girls') Leggings,Psst! Let's go and find that hidden treasure! ...


In [ ]:


print(df[df['Title'] == "Nike SB Dunk Low Pro Premium"]['Product Description'].iloc[0])


Pack your style—on your feet. Bringing a fresh twist to the iconic skate shoes, waxed canvas underlays and throwback branding nod to your favourite '90s backpack. Hidden details like the how-to guide on the sockliner remind you to go for it, while suede overlays and gum rubber sole let you transport yourself with A+ style.


# **Keep Learning!🔥**